### Imports

In [2]:
import os
import flywheel
import numpy as np
import pandas as pd
import nibabel as nib
from glob import glob
from os.path import join

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Custom helpers
import sys
sys.path.append("classes")
from helpers import *
from Model import Model
from Cloud import Cloud
from Button import Button
from Dicom import Dicom, DicomDir

### Features

In [7]:
num_features = (
    (len(voxel_radius(4)) * 2) + # 4 radius voxels (previous and next volume)
    (len(voxel_radius(4)) - len(voxel_radius(2))) + # 4 radius voxels not within 2 voxel radius for current volume
    2 + # time since (until) previous (next)
    1 + # time ratio between next voxels and previous
    2 + # time since last gonogo stimuli
    2 + # time since go/nogo bloack started
    6 + # time since faces stimuli
    6 + # time since faces block started
    2 + # time since wm stimuli
    2 + # time since wm block started
    2 + # time since guessing stimuli #FIXME
    2 + # time since guess block started #FIXME
    1 + # time since keypress (any)
    2 + # gender, age
    6 + # task identifier (is_gonogo, is_(non)conscious, is_wm, is_gambling, is_hairi)
    3 + # study identifier (conn, engage, rad)
    1 + # is_mb
    0
)
"Number of features:", num_features

('Number of features:', 776)

### Load objects

In [ ]:
cloud = Cloud()
model = Model()

### Load settings

In [5]:
tasks  = "config/tasks.json"
EPOCHS = 100

### Build model

In [17]:
def build_model(num_features):
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=(num_features, 1)),
        layers.Conv1D(activation=tf.nn.relu),
        layers.Dense(64, activation=tf.nn.relu),
        layers.AveragePooling1D(pool_size=2),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam()

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error']
    )
    return model

model = build_model(num_features)

TypeError: __init__() missing 2 required positional arguments: 'filters' and 'kernel_size'

In [ ]:
### Iterate over tasks
for task in tasks:
    task_info = tasks[task]
    num_vols  = task_info["vol"]
    if "project" in task_info:
        projects = task_info["project"]
    else:
        projects = ["rad", "connectome", "engage"]
        
    ### Iterate over projects
    for project in projects:
        time_sessions = project_config["time_sessions"]
        subjects = load_project_subjects(project)
        
        ### Iterate over time_session/subject
        for time_session in time_sessions:
            for subject in subjects:
                ### Set logger info
                logger.set(project=project, subject=subject, time_session=time_session, task=task)
            
                ### Load necessary data
                patient    = Patient(project, subject, time_session)
                dicom_path = cloud.download(project, subject, time_session, task)
                button     = Button(project, subject, time_session, task)
                
                ### Check all data exists
                if not _valid_patient(patient, dicom_path, button, logger):
                    continue
                    
                ### More setup
                fmri_data = DicomDir(dicom_path)
                fmri_data.cut_volumes(fmri_data.num_volumes - num_vols)
                
                ### Modeling
                model.load(model_path)
                model.run(patient, fmri_data, button)
                model.save(model_path)